# To Do

1. support function calling -- maybe inherit from working model or implement nesscary methods 
2. should get same output form from just using exisitng chat model class alone by mocking out the stream method 

In [1]:
# Import relevant functionality
# from langchain_anthropic import ChatAnthropic
from langchain_ollama import ChatOllama
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import HumanMessage, SystemMessage, ToolMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent


In [5]:
import os
import getpass

os.environ["TAVILY_API_KEY"] = "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"
# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

In [16]:
from langchain_terminal import CustomLLM
custom_llm = CustomLLM(server_url="http://127.0.0.1:7001/v1/completions")

# new endpoint -- check in with that 

# curretly running mock w tt-inferene server 
# how want agent to be implemented 
# have the agent working with the mock model
    # couple of bugs that I am  wokring through, like stop token 
# working from terminal 

# async for chunk in custom_llm.astream("hello", stop=["<|eot_id|>"]):
async for chunk in custom_llm.astream([HumanMessage(content="can you do tool calls!")]):
    print(chunk.content, end="|", flush=True)



|I|'d| be| happy| to| help| with| tool| calls| for| you|.| What| kind| of| tool| calls| would| you| like| me| to| assist| with|?| Would| you| like| me| to|:

|1|.| Generate| a| random| tool| call| for| you| to| practice| with|?
|2|.| Help| you| with| a| specific| tool| call| scenario| or| problem| you|'re| facing|?
|3|.| Practice| a| particular| tool| call| technique| or| skill| with| you|?

|Let| me| know|,| and| I|'ll| do| my| best| to| assist| you|!||

In [ ]:
async for event in custom_llm.astream_events("cat", version="v1"):
    print(event)

In [36]:
memory = MemorySaver()
search = TavilySearchResults(
    max_results=5,
    include_answer=True,
    include_raw_content=True,
    include_images=True)
tools = [search]
agent_executor = create_react_agent(custom_llm, tools, checkpointer=memory)
config = {"configurable": {"thread_id": "abc123"}}

In [37]:
async for event in agent_executor.astream_events(
    {"messages": [SystemMessage(content="You are a helpful assistant"), HumanMessage(content="where should I move from where I live currently if I want weather that is the same as where I am now?")]}, version="v1", config=config
):
    # print(event)
    kind = event["event"]
    if kind == "on_chain_start":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print(
                f"Starting agent: {event['name']} with input: {event['data'].get('input')}"
            )
    elif kind == "on_chain_end":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print()
            print("--")
            print(
                f"Done agent: {event['name']} with output: {event['data'].get('output')['output']}"
            )
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        # print("on model stream")
        if content:
            # Empty content in the context of OpenAI means
            # that the model is asking for a tool to be invoked.
            # So we only print non-empty content
            print(content, end="|")
    elif kind == "on_tool_start":
        print("--")
        print(
            f"Starting tool: {event['name']} with inputs: {event['data'].get('input')}"
        )
    elif kind == "on_tool_end":
        print(f"Done tool: {event['name']}")
        print(f"Tool output was: {event['data'].get('output')}")
        print("--")

--
Starting tool: tavily_search_results_json with inputs: {'query': 'current weather conditions'}


Done tool: tavily_search_results_json
Tool output was: content='[{"url": "https://www.weatherapi.com/", "content": "{\'location\': {\'name\': \\"Modjadje\'s Location\\", \'region\': \'Limpopo\', \'country\': \'South Africa\', \'lat\': -23.6167, \'lon\': 30.3333, \'tz_id\': \'Africa/Johannesburg\', \'localtime_epoch\': 1733859942, \'localtime\': \'2024-12-10 21:45\'}, \'current\': {\'last_updated_epoch\': 1733859900, \'last_updated\': \'2024-12-10 21:45\', \'temp_c\': 24.4, \'temp_f\': 75.9, \'is_day\': 0, \'condition\': {\'text\': \'Clear\', \'icon\': \'//cdn.weatherapi.com/weather/64x64/night/113.png\', \'code\': 1000}, \'wind_mph\': 4.5, \'wind_kph\': 7.2, \'wind_degree\': 217, \'wind_dir\': \'SW\', \'pressure_mb\': 1014.0, \'pressure_in\': 29.94, \'precip_mm\': 0.0, \'precip_in\': 0.0, \'humidity\': 42, \'cloud\': 5, \'feelslike_c\': 25.1, \'feelslike_f\': 77.1, \'windchill_c\': 24.4, \'windchill_f\': 75.9, \'heatindex_c\': 25.1, \'heatindex_f\': 77.1, \'dewpoint_c\': 10.5, \'dewpoi

In [20]:
config = {"configurable": {"thread_id": "abc123"}} 
async for event in agent_executor.astream_events(
    {"messages": [SystemMessage(content="You are a helpful assistant"), HumanMessage(content="where should I move from where I live currently if I want weather that is the same as where I am now?")]}, version="v1", config=config
):
    print(event)
    kind = event["event"]
    if kind == "on_chain_start":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print(
                f"Starting agent: {event['name']} with input: {event['data'].get('input')}"
            )
    elif kind == "on_chain_end":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print()
            print("--")
            print(
                f"Done agent: {event['name']} with output: {event['data'].get('output')['output']}"
            )
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        print("on model stream")
        if content:
            # Empty content in the context of OpenAI means
            # that the model is asking for a tool to be invoked.
            # So we only print non-empty content
            print(content, end="|")
    elif kind == "on_tool_start":
        print("--")
        print(
            f"Starting tool: {event['name']} with inputs: {event['data'].get('input')}"
        )
    elif kind == "on_tool_end":
        print(f"Done tool: {event['name']}")
        print(f"Tool output was: {event['data'].get('output')}")
        print("--")

{'event': 'on_chain_start', 'run_id': '5bc687cd-da07-40ea-85a4-29b06b4842be', 'name': 'LangGraph', 'tags': [], 'metadata': {'thread_id': 'abc123'}, 'data': {'input': {'messages': [SystemMessage(content='You are a helpful assistant', additional_kwargs={}, response_metadata={}), HumanMessage(content='where should I move from where I live currently if I want weather that is the same as where I am now?', additional_kwargs={}, response_metadata={})]}}, 'parent_ids': []}
{'event': 'on_chain_start', 'name': '__start__', 'run_id': '16255ce7-df7a-454a-96bc-a31673472a78', 'tags': ['graph:step:5', 'langsmith:hidden', 'langsmith:hidden'], 'metadata': {'thread_id': 'abc123', 'langgraph_step': 5, 'langgraph_node': '__start__', 'langgraph_triggers': ['__start__'], 'langgraph_path': ('__pregel_pull', '__start__'), 'langgraph_checkpoint_ns': '__start__:8df7355e-7f6f-ce1b-86eb-485376a5e36c'}, 'data': {'input': {'messages': [SystemMessage(content='You are a helpful assistant', additional_kwargs={}, respo

In [21]:
os.environ["GROQ_API_KEY"] = "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"

from langchain_groq import ChatGroq

model = ChatGroq(model="llama3-8b-8192")

In [33]:
os.environ["GROQ_API_KEY"] = "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"
memory = MemorySaver()
llm = ChatGroq(model="llama3-8b-8192")
search = TavilySearchResults(
    max_results=5,
    include_answer=True,
    include_raw_content=True,
    include_images=True)

tools = [search]
agent_executor = create_react_agent(llm, tools, checkpointer=memory)
config = {"configurable": {"thread_id": "cde456"}} 

In [35]:
async for event in agent_executor.astream_events(
    {"messages": [HumanMessage(content="what is the weather in toronto right now")]}, version="v2", config=config
):
    kind = event["event"]
    if kind == "on_chain_start":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print(
                f"Starting agent: {event['name']} with input: {event['data'].get('input')}"
            )
    elif kind == "on_chain_end":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print()
            print("--")
            print(
                f"Done agent: {event['name']} with output: {event['data'].get('output')['output']}"
            )
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            # Empty content in the context of OpenAI means
            # that the model is asking for a tool to be invoked.
            # So we only print non-empty content
            print(content, end="|")
    elif kind == "on_tool_start":
        print("--")
        print(
            f"Starting tool: {event['name']} with inputs: {event['data'].get('input')}"
        )
    elif kind == "on_tool_end":
        print(f"Done tool: {event['name']}")
        print(f"Tool output was: {event['data'].get('output')}")
        print("--")



--
Starting tool: tavily_search_results_json with inputs: {'query': 'Current weather in Toronto'}
Done tool: tavily_search_results_json
Tool output was: content='[{"url": "https://www.weatherapi.com/", "content": "{\'location\': {\'name\': \'Toronto\', \'region\': \'Ontario\', \'country\': \'Canada\', \'lat\': 43.6667, \'lon\': -79.4167, \'tz_id\': \'America/Toronto\', \'localtime_epoch\': 1733859873, \'localtime\': \'2024-12-10 14:44\'}, \'current\': {\'last_updated_epoch\': 1733859000, \'last_updated\': \'2024-12-10 14:30\', \'temp_c\': 7.7, \'temp_f\': 45.9, \'is_day\': 1, \'condition\': {\'text\': \'Sunny\', \'icon\': \'//cdn.weatherapi.com/weather/64x64/day/113.png\', \'code\': 1000}, \'wind_mph\': 4.0, \'wind_kph\': 6.5, \'wind_degree\': 159, \'wind_dir\': \'SSE\', \'pressure_mb\': 1013.0, \'pressure_in\': 29.9, \'precip_mm\': 0.0, \'precip_in\': 0.0, \'humidity\': 89, \'cloud\': 0, \'feelslike_c\': 6.7, \'feelslike_f\': 44.0, \'windchill_c\': 3.8, \'windchill_f\': 38.8, \'heatin